<a href="https://colab.research.google.com/github/kaonei02/linebot-gemini-python/blob/main/GDP_AGGR_ALAN_0302.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

調整紀錄
- 性別居住地 高所得低所得 null處理
- 人壽繳費方式 null處理

## 模組導入

In [ ]:
import pymongo
from pymongo import MongoClient
from itertools import chain
from itertools import product

import os
import time
from collections import defaultdict

ModuleNotFoundError: No module named 'pymongo'

## 連線設定

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://<username>:<password>@cluster0.mttxvlm.mongodb.net/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

ModuleNotFoundError: No module named 'pymongo'

## 參數設定

In [ ]:
db = client['testforgdp']

collection_name = db["test_customer_dim"]
GDP_CUSTOMER_AGGREGATION = "test_customer_aggr"

calculate_month = 202310

# 設定公司迴圈數量
company_count = range(1, 6)
identity_count = range(1, 3)
status_count = range(1, 4)

# 設定產品資訊
products_info = [
    {"type_id": 1, "product_id": [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]},
    {"type_id": 2, "product_id": [601, 602, 603]},
    {"type_id": 3, "product_id": [701, 702]},
    {"type_id": 4, "product_id": [101, 102, 103, 104, 105, 106, 107, 108, 109]},
    {"type_id": 5, "product_id": [201, 202, 203, 204, 205, 206, 207, 208, 209]},
    {"type_id": 6, "product_id": [1, 2, 3, 4]},
    {"type_id": 7, "product_id": [""]},
    {"type_id": 8, "product_id": [1, 2, 3, 4, 5, 6]},
    {"type_id": 9, "product_id": [1, 2, 3, 4, 5]}
]

scope_list = ['loan_limit_scope','loan_interest_rate_scope','loan_balance_scope']

## 指標計算 def aggregate_data

In [ ]:
################################################# pipeline #################################################
def aggregate_data(collection_name):
    customer_quantity_pipeline = [
        condition_code,
        {
            # 計算上游階段結果的 doc 總數
            "$count": "all_customer_quantity"
        }
    ]

    gender_pipeline = [
        condition_code,
        {
            "$group": {
                '_id': {
                    '$ifNull': [
                        '$gender', 0
                    ]
                },
                "customer_quantity": { "$sum": 1 }
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    residence_pipeline = [
        condition_code,
        {
            "$group": {
                '_id': {
                    '$ifNull': [
                        '$residence', 0
                    ]
                },
                "customer_quantity": { "$sum": 1 }
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    age_scope_pipeline = [
        condition_code,
        {
            "$group": {
                "_id": "$age_scope",
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    annual_income_scope_pipeline = [
        condition_code,
        {
            "$group": {
                "_id": "$annual_income_scope",
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    annual_income_low_scope_pipeline = [
        condition_code,
        {
            "$group": {
                "_id": "$anunal_income_low_scope", ## 年收入區間(低)
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    annual_income_high_scope_pipeline = [
        condition_code,
        {
            "$group": {
                "_id": "$anunal_income_high_scope", ## 年收入區間(高)
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    age_gender_pipeline = [
        condition_code,
        {
            "$group": {
                "_id": {
                    "age_scope_id": "$age_scope",
                    "gender": "$gender"
                },
                # 先算出每個性別、年齡切分下的客戶數量
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$group": {
                "_id": "$_id.age_scope_id",
                "customer_quantity": {
                    # push: 將該性別及客戶數量 array 分別添加到該 age_scope 下的性別及客戶數量欄位
                    "$push": {
                        "gender": "$_id.gender",
                        "quantity": "$customer_quantity"
                    }
                }
            }
        },
        {
            # 以下流成順序：
            # project: 重組輸出格式，預設已包含 _id (在此為 age_scope_id) 欄位，因此只需對 customer_quantity 做計算
            "$project": {
                "customer_quantity": {
                    # 因 map 出來的結果會是一個 array，把他以 arrayToObject 轉成 object (同 struct, dict)
                    "$arrayToObject": {
                        # 以 map 對 customer_quantity 這個 input array 中的每個 item 做映射，產生新的 key-value pair
                        "$map": {
                            "input": "$customer_quantity",
                            # in: 針對 input 執行處理的表達式 (執行規則)
                            "in": {
                                # k: key, 以判斷式判斷 key 值，如當前 doc 的 gender 欄位為 1 則是 man; 為 2 則是 woman
                                "k": {
                                    "$cond": {
                                        "if": { "$eq": ["$$this.gender", 1] },
                                        "then": "man",
                                        "else": "woman"
                                    }
                                },
                                # v: value, 當前 doc 的 quantity 欄位
                                "v": "$$this.quantity"
                            }
                        }
                    }
                }
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    platform_pipeline = [
        condition_code,
        {
            '$unwind': '$platform_activity_status'
        }, {
            '$group': {
                '_id': {
                    'platform_id': '$platform_activity_status.platform_id',
                    'activity_status': '$platform_activity_status.activity_status'
                },
                'customer_quantity': {
                    '$sum': 1
                }
            }
        }, {
            '$project': {
                'platform_id': '$_id.platform_id',
                'new_customer_quantity': {
                    '$cond': {
                        'if': {
                            '$eq': [
                                '$_id.activity_status', 1
                            ]
                        },
                        'then': '$customer_quantity',
                        'else': None
                    }
                },
                'existing_customer_quantity': {
                    '$cond': {
                        'if': {
                            '$eq': [
                                '$_id.activity_status', 2
                            ]
                        },
                        'then': '$customer_quantity',
                        'else': None
                    }
                },
                '_id': 0
            }
        }, {
            '$group': {
                '_id': '$platform_id',
                'new_customer_quantity': {
                    '$sum': '$new_customer_quantity'
                },
                'existing_customer_quantity': {
                    '$sum': '$existing_customer_quantity'
                }
            }
        }, {
            '$sort': {
                'platform_id': 1
            }
        }
    ]

    bank_account_type_pipeline = [
        condition_code,
        {
            "$unwind": "$bank_account_type_list"
        },
        {
            "$group": {
                "_id": "$bank_account_type_list",
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    investment_product_pipeline = [
        condition_code,
        {
            "$unwind": "$investment_product_list"
        },
        {
            "$group": {
                "_id": "$investment_product_list",
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    credit_card_number_pipeline = [
        condition_code,
        {
            "$unwind": "$credit_card_info"
        },
        {
            "$group": {
                "_id": "$credit_card_info.credit_card_number_scope",
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

    credit_card_limit_pipeline = [
        condition_code,
        {
            "$unwind": "$credit_card_info"
        },
        {
            "$group": {
                "_id": "$credit_card_info.credit_card_limit_scope",
                "customer_quantity": {"$sum": 1}
            }
        },
        {
            "$sort": {"_id": 1}
        }
    ]

##unsecured_loan_product_pipeline
    unsecured_loan_product_pipeline_sum = [
        condition_code,
            {
                "$unwind": "$loan_product_info.unsecured_loan_product_list"
            },
            {
                "$group": {
                    "_id": {"loan_product_id": "$loan_product_info.unsecured_loan_product_list.loan_product_id",
                            "loan_limit_scope":"$loan_product_info.unsecured_loan_product_list.loan_limit_scope"},
                    "quantity": {"$sum": 1}
                }
            },
            {
                "$group": {
                    "_id": "$_id.loan_product_id",
                    "customer_quantity": {"$sum": "$quantity"}
                }
            },
            {
                "$sort": {"_id": 1}
            }
        ]

##secured_loan_product_pipeline 先insurance 後bank
    life_insurance_loan_product_pipeline_sum = [
        condition_code,
            {
                "$unwind": "$loan_product_info.secured_loan_product_list.life_insurance_loan_product_list"
            },
            {
                "$group": {
                    "_id": {"loan_product_id": "$loan_product_info.secured_loan_product_list.life_insurance_loan_product_list.loan_product_id",
                            "loan_limit_scope":"$loan_product_info.secured_loan_product_list.life_insurance_loan_product_list.loan_limit_scope"},
                    "quantity": {"$sum": 1}
                }
            },
            {
                "$group": {
                    "_id": "$_id.loan_product_id",
                    "customer_quantity": {"$sum": "$quantity"}
                }
            },
            {
                "$sort": {"_id": 1}
            }
        ]

    bank_loan_product_pipeline_sum = [
        condition_code,
            {
                "$unwind": "$loan_product_info.secured_loan_product_list.bank_loan_product_list"
            },
            {
                "$group": {
                    "_id": {"loan_product_id": "$loan_product_info.secured_loan_product_list.bank_loan_product_list.loan_product_id",
                            "loan_limit_scope":"$loan_product_info.secured_loan_product_list.bank_loan_product_list.loan_limit_scope"},
                    "quantity": {"$sum": 1}
                }
            },
            {
                "$group": {
                    "_id": "$_id.loan_product_id",
                    "customer_quantity": {"$sum": "$quantity"}
                }
            },
            {
                "$sort": {"_id": 1}
            }
        ]

## insurance_product_info
    life_insurance_valid_insurance_num_pipeline = [
        condition_code,
        {
            '$match': {
                'insurance_product_info.life_insurance_product': {
                    '$exists': True
                }
            }
        }, {
            '$group': {
                '_id': None,
                'valid_insurance_num': {
                    '$sum': 1
                }
            }
        }, {
            '$project': {
                '_id': 0,
            }
        }
    ]
    # 人壽 payment_type 多一個 $match stage 把 type_id=null拿掉
    life_insurance_payment_type_pipeline = [
        condition_code,
        {
            '$unwind': '$insurance_product_info.life_insurance_product.insurance_product_list'
        }, {
            '$unwind': '$insurance_product_info.life_insurance_product.insurance_product_list.payment_type'
        }, {
            '$match': {
                'insurance_product_info.life_insurance_product.insurance_product_list.payment_type.type_id': {
                    '$ne': None
                }
            }
        }, {
            '$group': {
                '_id': {
                    'insurance_product_id': '$insurance_product_info.life_insurance_product.insurance_product_list.insurance_product_id',
                                'type_id': '$insurance_product_info.life_insurance_product.insurance_product_list.payment_type.type_id'
                },
                'policy_quantity': {
                    '$sum': '$insurance_product_info.life_insurance_product.insurance_product_list.payment_type.policy_quantity'
                },
                'customer_quantity': {
                    '$sum': 1
                }
            }
        }, {
            '$group': {
                '_id': '$_id.insurance_product_id',
                'payment_type': {
                    '$push': {
                        'type_id': '$_id.type_id',
                        'policy_quantity': '$policy_quantity',
                        'customer_quantity': '$customer_quantity'
                    }
                }
            }
        }, {
            '$unwind': '$payment_type'
        }, {
            '$sort': {
                '_id': 1,
                'payment_type': 1
            }
        }, {
            '$group': {
                '_id': '$_id',
                'payment_type': {
                    '$push': {
                        'type_id': '$payment_type.type_id',
                        'policy_quantity': '$payment_type.policy_quantity',
                        'customer_quantity': '$payment_type.customer_quantity'
                    }
                }
            }
        }
    ]
    life_insurance_purchase_channel_pipeline = [
        condition_code,
        {
            '$unwind': '$insurance_product_info.life_insurance_product.insurance_product_list'
        }, {
            '$unwind': '$insurance_product_info.life_insurance_product.insurance_product_list.purchase_channel'
        }, {
            '$group': {
                '_id': {
                    'insurance_product_id': '$insurance_product_info.life_insurance_product.insurance_product_list.insurance_product_id',
                                'channel_id': '$insurance_product_info.life_insurance_product.insurance_product_list.purchase_channel.channel_id'
                },
                'policy_quantity': {
                    '$sum': '$insurance_product_info.life_insurance_product.insurance_product_list.purchase_channel.policy_quantity'
                },
                'customer_quantity': {
                    '$sum': 1
                }
            }
        }, {
            '$group': {
                '_id': '$_id.insurance_product_id',
                'purchase_channel': {
                    '$push': {
                        'channel_id': '$_id.channel_id',
                        'policy_quantity': '$policy_quantity',
                        'customer_quantity': '$customer_quantity'
                    }
                }
            }
        }, {
            '$unwind': '$purchase_channel'
        }, {
            '$sort': {
                '_id': 1,
                'purchase_channel': 1
            }
        }, {
            '$group': {
                '_id': '$_id',
                'purchase_channel': {
                    '$push': {
                        'channel_id': '$purchase_channel.channel_id',
                        'policy_quantity': '$purchase_channel.policy_quantity',
                        'customer_quantity': '$purchase_channel.customer_quantity'
                    }
                }
            }
        }
    ]
    property_insurance_valid_insurance_num_pipeline = [
        condition_code,
        {
            '$match': {
                'insurance_product_info.property_insurance_product': {
                    '$exists': True
                }
            }
        }, {
            '$group': {
                '_id': None,
                'valid_insurance_num': {
                    '$sum': 1
                }
            }
        }, {
            '$project': {
                '_id': 0,
            }
        }
    ]
    property_insurance_payment_type_pipeline = [
        condition_code,
        {
            '$unwind': '$insurance_product_info.property_insurance_product.insurance_product_list'
        }, {
            '$unwind': '$insurance_product_info.property_insurance_product.insurance_product_list.payment_type'
        }, {
            '$group': {
                '_id': {
                    'insurance_product_id': '$insurance_product_info.property_insurance_product.insurance_product_list.insurance_product_id',
                    'type_id': '$insurance_product_info.property_insurance_product.insurance_product_list.payment_type.type_id'
                },
                'policy_quantity': {
                    '$sum': '$insurance_product_info.property_insurance_product.insurance_product_list.payment_type.policy_quantity'
                },
                'customer_quantity': {
                    '$sum': 1
                }
            }
        }, {
            '$group': {
                '_id': '$_id.insurance_product_id',
                'payment_type': {
                    '$push': {
                        'type_id': '$_id.type_id',
                        'policy_quantity': '$policy_quantity',
                        'customer_quantity': '$customer_quantity'
                    }
                }
            }
        }, {
            '$unwind': '$payment_type'
        }, {
            '$sort': {
                '_id': 1,
                'payment_type': 1
            }
        }, {
            '$group': {
                '_id': '$_id',
                'payment_type': {
                    '$push': {
                        'type_id': '$payment_type.type_id',
                        'policy_quantity': '$payment_type.policy_quantity',
                        'customer_quantity': '$payment_type.customer_quantity'
                    }
                }
            }
        }
    ]
    property_insurance_purchase_channel_pipeline = [
        condition_code,
        {
            '$unwind': '$insurance_product_info.property_insurance_product.insurance_product_list'
        }, {
            '$unwind': '$insurance_product_info.property_insurance_product.insurance_product_list.purchase_channel'
        }, {
            '$group': {
                '_id': {
                    'insurance_product_id': '$insurance_product_info.property_insurance_product.insurance_product_list.insurance_product_id',
                    'channel_id': '$insurance_product_info.property_insurance_product.insurance_product_list.purchase_channel.channel_id'
                },
                'policy_quantity': {
                    '$sum': '$insurance_product_info.property_insurance_product.insurance_product_list.purchase_channel.policy_quantity'
                },
                'customer_quantity': {
                    '$sum': 1
                }
            }
        }, {
            '$group': {
                '_id': '$_id.insurance_product_id',
                'purchase_channel': {
                    '$push': {
                        'channel_id': '$_id.channel_id',
                        'policy_quantity': '$policy_quantity',
                        'customer_quantity': '$customer_quantity'
                    }
                }
            }
        }, {
            '$unwind': '$purchase_channel'
        }, {
            '$sort': {
                '_id': 1,
                'purchase_channel': 1
            }
        }, {
            '$group': {
                '_id': '$_id',
                'purchase_channel': {
                    '$push': {
                        'channel_id': '$purchase_channel.channel_id',
                        'policy_quantity': '$purchase_channel.policy_quantity',
                        'customer_quantity': '$purchase_channel.customer_quantity'
                    }
                }
            }
        }
    ]

## 匯總數據
    summary_pipeline = [
        condition_code,
        {
            '$facet': {
                'pipeline1': [
                    {
                        '$match': {
                            'loan_product_info.unsecured_loan_product_list': {
                                '$exists': True
                            }
                        }
                    }, {
                        '$group': {
                            '_id': None,
                            'unsecured_loan': {
                                '$sum': 1
                            }
                        }
                    }
                ],
                'pipeline2': [
                    {
                        '$match': {
                            'loan_product_info.secured_loan_product_list': {
                                '$exists': True
                            }
                        }
                    }, {
                        '$group': {
                            '_id': None,
                            'secured_loan': {
                                '$sum': 1
                            }
                        }
                    }
                ],
                'pipeline3': [
                    {
                        '$match': {
                            'loan_product_info.secured_loan_product_list.bank_loan_product_list': {
                                '$exists': True
                            }
                        }
                    }, {
                        '$group': {
                            '_id': None,
                            'secured_loan_bank': {
                                '$sum': 1
                            }
                        }
                    }
                ],
                'pipeline4': [
                    {
                        '$match': {
                            'loan_product_info.secured_loan_product_list.life_insurance_loan_product_list': {
                                '$exists': True
                            }
                        }
                    }, {
                        '$group': {
                            '_id': None,
                            'secured_loan_insurance': {
                                '$sum': 1
                            }
                        }
                    }
                ],
                'pipeline5': [
                    {
                        '$match': {
                            'insurance_product_info.life_insurance_product': {
                                '$exists': True
                            }
                        }
                    }, {
                        '$group': {
                            '_id': None,
                            'life_insurance': {
                                '$sum': 1
                            }
                        }
                    }
                ],
                'pipeline6': [
                    {
                        '$match': {
                            'insurance_product_info.property_insurance_product': {
                                '$exists': True
                            }
                        }
                    }, {
                        '$group': {
                            '_id': None,
                            'property_insurance': {
                                '$sum': 1
                            }
                        }
                    }
                ]
            }
        }, {
            '$project': {
                '_id': 0,
                'summary_customer_quantity': {
                    '$arrayToObject': {
                        '$concatArrays': [
                            [
                                {
                                    'k': 'unsecured_loan',
                                    'v': {
                                        '$ifNull': [
                                            { '$arrayElemAt': ['$pipeline1.unsecured_loan', 0] },
                                            0
                                        ]
                                    }
                                }, {
                                    'k': 'secured_loan',
                                    'v': {
                                        '$ifNull': [
                                            { '$arrayElemAt': ['$pipeline2.secured_loan', 0] },
                                            0
                                        ]
                                    }
                                }, {
                                    'k': 'secured_loan_bank',
                                    'v': {
                                        '$ifNull': [
                                            { '$arrayElemAt': ['$pipeline3.secured_loan_bank', 0] },
                                            0
                                        ]
                                    }
                                }, {
                                    'k': 'secured_loan_insurance',
                                    'v': {
                                        '$ifNull': [
                                            { '$arrayElemAt': ['$pipeline4.secured_loan_insurance', 0] },
                                            0
                                        ]
                                    }
                                }, {
                                    'k': 'life_insurance_product',
                                    'v': {
                                        '$ifNull': [
                                            { '$arrayElemAt': ['$pipeline5.life_insurance', 0] },
                                            0
                                        ]
                                    }
                                }, {
                                    'k': 'property_insurance',
                                    'v': {
                                        '$ifNull': [
                                            { '$arrayElemAt': ['$pipeline6.property_insurance', 0] },
                                            0
                                        ]
                                    }
                                }
                            ]
                        ]
                    }
                }
            }
        }
    ]





################################################# 指標計算 #################################################

## 客戶總數
    ## 如果完全沒有符合的 document，則填0否則會報錯
    try:
        customer_quantity_sum = list(collection_name.aggregate(customer_quantity_pipeline))[0]["all_customer_quantity"]
    except IndexError:
        customer_quantity_sum = 0

## 性別
    gender_result = list(collection_name.aggregate(gender_pipeline))
    gender_list = [{"gender_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                   for r in gender_result]
## 居住地
    residence_result = list(collection_name.aggregate(residence_pipeline))
    residence_list = [{"residence_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                      for r in residence_result]
## 年齡區間      ## 不要搞混 age 及 age_scope
    age_scope_result = list(collection_name.aggregate(age_scope_pipeline))
    age_list = [{"age_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                      for r in age_scope_result]
## 年收入區間
    annual_income_scope_result = list(collection_name.aggregate(annual_income_scope_pipeline))
    annual_income_list = [{"annual_income_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                                for r in annual_income_scope_result]
## 年收入區間(低)
    annual_income_low_scope_result = list(collection_name.aggregate(annual_income_low_scope_pipeline))
    annual_income_low_list = [{"annual_income_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                                for r in annual_income_low_scope_result]
## 年收入區間(高)
    annual_income_high_scope_result = list(collection_name.aggregate(annual_income_high_scope_pipeline))
    annual_income_high_list = [{"annual_income_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                                for r in annual_income_high_scope_result]
## 年齡區間及性別 (金字塔圖)
    age_gender_result = list(collection_name.aggregate(age_gender_pipeline))
    age_gender_list = [{"age_id": r["_id"], "customer_quantity_by_gender": r["customer_quantity"]}
                       for r in age_gender_result]
## 平台
    platform_result = list(collection_name.aggregate(platform_pipeline))
    platform_list = [{"platform_id": r["_id"], "new_customer_quantity": r["new_customer_quantity"], "existing_customer_quantity": r["existing_customer_quantity"]}
                     for r in platform_result]
## 銀行帳戶
    bank_account_type_result = list(collection_name.aggregate(bank_account_type_pipeline))
    bank_account_type_list = [{"bank_account_type": r["_id"], "customer_quantity": r["customer_quantity"]}
                              for r in bank_account_type_result]
## 信用卡張數
    credit_card_number_result = list(collection_name.aggregate(credit_card_number_pipeline))
    credit_card_number_list = [{"credit_card_number_scope": r["_id"], "customer_quantity": r["customer_quantity"]}
                              for r in credit_card_number_result]
## 信用卡額度
    credit_card_limit_result = list(collection_name.aggregate(credit_card_limit_pipeline))
    credit_card_limit_list = [{"credit_card_limit_scope": r["_id"], "customer_quantity": r["customer_quantity"]}
                              for r in credit_card_limit_result]
## 理財商品
    investment_product_result = list(collection_name.aggregate(investment_product_pipeline))
    investment_product_list = [{"investment_product_id": r["_id"], "customer_quantity": r["customer_quantity"]}
                              for r in investment_product_result]
## 無擔保商品
    unsecured_loan_product_list = []
    unsecured_loan_product_list = unsecured_loan_product_list + list(collection_name.aggregate(unsecured_loan_product_pipeline_sum))
    # 加上三個 scope
    for scope in scope_list:
        unsecured_loan_product_pipeline = [
            condition_code,
            {
                "$unwind": "$loan_product_info.unsecured_loan_product_list"
            },
            {
                "$group": {
                    "_id": {"loan_product_id": "$loan_product_info.unsecured_loan_product_list.loan_product_id",
                    f"{scope}":f"$loan_product_info.unsecured_loan_product_list.{scope}"},
                    "customer_quantity": {"$sum": 1}
                }
            },
            {
                "$sort": {"_id": 1}
            },
            {
                "$group": {
                    "_id": "$_id.loan_product_id",
                    "customer_quantity": {
                        # push: 將該scope及數量 array
                        "$push": {
                            "scope":f"$_id.{scope}",
                            "quantity": "$customer_quantity"
                        }
                    }
                }
            },
            {
                "$project": {
                    "filter_scope": {
                        "$map": {
                            "input": {"$range": [1, 12]},  # 這裡的 [0, 5] 是區間的範圍，你可以根據實際需求調整
                            "as": "i",
                            "in": {
                                "$cond": {
                                    "if": {
                                        "$in": ["$$i", {"$map": {"input": "$customer_quantity", "as": "c", "in": "$$c.scope"}}]
                                    },
                                    "then": {
                                        "$arrayElemAt": [
                                            {
                                                "$filter": {
                                                    "input": "$customer_quantity",
                                                    "as": "c",
                                                    "cond": {"$eq": ["$$c.scope", "$$i"]}
                                                }
                                            },
                                            0
                                        ]
                                    },
                                    "else": {"scope": "$$i", "quantity": 0}
                                }
                            }
                        }
                    }
                }
            },
            {
                "$project": {
                  f"{scope}": {
                    "$map": {
                      "input": "$filter_scope",
                      #"as": "test",
                      "in": "$$this.quantity"
                    }
                  }
                }
              }
        ]
        unsecured_loan_product_list = unsecured_loan_product_list + list(collection_name.aggregate(unsecured_loan_product_pipeline))
    # unsecured_loan_product_list
    grouped_data = defaultdict(dict)
    for item in unsecured_loan_product_list:
        id_value = item['_id']
        item.pop('_id', None)  # 删除'i鍵，防止覆蓋
        grouped_data[id_value].update(item)
    # 將結果轉換為列表
    unsecured_loan_product_list = [{'loan_product_id': key, **value} for key, value in grouped_data.items()]

## 有擔保商品
## 有擔保商品 - 人壽
    life_insurance_loan_product_list = []
    life_insurance_loan_product_list = life_insurance_loan_product_list + list(collection_name.aggregate(life_insurance_loan_product_pipeline_sum))
    # 加上三個 scope
    for scope in scope_list:
        secured_loan_product_pipeline = [
            condition_code,
            {
                "$unwind": "$loan_product_info.secured_loan_product_list.life_insurance_loan_product_list"
            },
            {
                "$group": {
                    "_id": {"loan_product_id": "$loan_product_info.secured_loan_product_list.life_insurance_loan_product_list.loan_product_id",
                    f"{scope}":f"$loan_product_info.secured_loan_product_list.life_insurance_loan_product_list.{scope}"},
                    "customer_quantity": {"$sum": 1}
                }
            },
            {
                "$sort": {"_id": 1}
            },
            {
                "$group": {
                    "_id": "$_id.loan_product_id",
                    "customer_quantity": {
                        # push: 將該scope及數量 array
                        "$push": {
                            "scope":f"$_id.{scope}",
                            "quantity": "$customer_quantity"
                        }
                    }
                }
            },
            {
                "$project": {
                    "filter_scope": {
                        "$map": {
                            "input": {"$range": [1, 12]},  # 這裡的 [0, 5] 是區間的範圍，你可以根據實際需求調整
                            "as": "i",
                            "in": {
                                "$cond": {
                                    "if": {
                                        "$in": ["$$i", {"$map": {"input": "$customer_quantity", "as": "c", "in": "$$c.scope"}}]
                                    },
                                    "then": {
                                        "$arrayElemAt": [
                                            {
                                                "$filter": {
                                                    "input": "$customer_quantity",
                                                    "as": "c",
                                                    "cond": {"$eq": ["$$c.scope", "$$i"]}
                                                }
                                            },
                                            0
                                        ]
                                    },
                                    "else": {"scope": "$$i", "quantity": 0}
                                }
                            }
                        }
                    }
                }
            },
            {
                "$project": {
                  f"{scope}": {
                    "$map": {
                      "input": "$filter_scope",
                      #"as": "test",
                      "in": "$$this.quantity"
                    }
                  }
                }
              }
        ]
        life_insurance_loan_product_list = life_insurance_loan_product_list + list(collection_name.aggregate(secured_loan_product_pipeline))
    # life_insurance_loan_product_list
    grouped_data = defaultdict(dict)
    for item in life_insurance_loan_product_list:
        id_value = item['_id']
        item.pop('_id', None)  # 删除'i鍵，防止覆蓋
        grouped_data[id_value].update(item)
    # 將結果轉換為列表
    life_insurance_loan_product_list_result = [{'loan_product_id': key, **value} for key, value in grouped_data.items()]

## 有擔保商品 - 銀行
    bank_loan_product_list = []
    bank_loan_product_list = bank_loan_product_list + list(collection_name.aggregate(bank_loan_product_pipeline_sum))
    for scope in scope_list:
        bank_loan_product_pipeline = [
            condition_code,
            {
                "$unwind": "$loan_product_info.secured_loan_product_list.bank_loan_product_list"
            },
            {
                "$group": {
                    "_id": {"loan_product_id": "$loan_product_info.secured_loan_product_list.bank_loan_product_list.loan_product_id",
                    f"{scope}":f"$loan_product_info.secured_loan_product_list.bank_loan_product_list.{scope}"},
                    "customer_quantity": {"$sum": 1}
                }
            },
            {
                "$sort": {"_id": 1}
            },
            {
                "$group": {
                    "_id": "$_id.loan_product_id",
                    "customer_quantity": {
                        # push: 將該scope及數量 array
                        "$push": {
                            "scope":f"$_id.{scope}",
                            "quantity": "$customer_quantity"
                        }
                    }
                }
            },
            {
                "$project": {
                    "filter_scope": {
                        "$map": {
                            "input": {"$range": [1, 12]},  # 這裡的 [] 是區間的範圍，你可以根據實際需求調整
                            "as": "i",
                            "in": {
                                "$cond": {
                                    "if": {
                                        "$in": ["$$i", {"$map": {"input": "$customer_quantity", "as": "c", "in": "$$c.scope"}}]
                                    },
                                    "then": {
                                        "$arrayElemAt": [
                                            {
                                                "$filter": {
                                                    "input": "$customer_quantity",
                                                    "as": "c",
                                                    "cond": {"$eq": ["$$c.scope", "$$i"]}
                                                }
                                            },
                                            0
                                        ]
                                    },
                                    "else": {"scope": "$$i", "quantity": 0}
                                }
                            }
                        }
                    }
                }
            },
            {
                "$project": {
                  f"{scope}": {
                    "$map": {
                      "input": "$filter_scope",
                      #"as": "test",
                      "in": "$$this.quantity"
                    }
                  }
                }
              }
        ]
        bank_loan_product_list = bank_loan_product_list + list(collection_name.aggregate(bank_loan_product_pipeline))
    # bank_loan_product_list
    grouped_data = defaultdict(dict)
    for item in bank_loan_product_list:
        id_value = item['_id']
        item.pop('_id', None)  # 删除'i鍵，防止覆蓋
        grouped_data[id_value].update(item)
    # 將結果轉換為列表
    bank_loan_product_list_result = [{'loan_product_id': key, **value} for key, value in grouped_data.items()]

    # secured loan 合併 #
    secured_loan_product = {}
    secured_loan_product["bank_loan_product_list"] = bank_loan_product_list_result
    secured_loan_product["life_insurance_loan_product_list"] = life_insurance_loan_product_list_result

## 保險
## 保險 - 人壽
    life_insurance_product = {}
    life_insurance_valid_insurance_num = list(collection_name.aggregate(life_insurance_valid_insurance_num_pipeline))
    ### 如果完全沒有符合的 document，則跳過
    try:
        life_insurance_product['valid_insurance_num'] = life_insurance_valid_insurance_num[0]['valid_insurance_num']
    except IndexError:
        life_insurance_product['valid_insurance_num'] = 0
    insurance_product_list=[]
    insurance_product_list= insurance_product_list + list(collection_name.aggregate(life_insurance_payment_type_pipeline))
    insurance_product_list= insurance_product_list + list(collection_name.aggregate(life_insurance_purchase_channel_pipeline))
    # grouping
    grouped_data = defaultdict(dict)
    for item in insurance_product_list:
        id_value = item['_id']
        item.pop('_id', None)  # 删除'i鍵，防止覆蓋
        grouped_data[id_value].update(item)
    insurance_product_list = [{'insurance_product_id': key, **value} for key, value in grouped_data.items()]
    life_insurance_product['insurance_product_list']= insurance_product_list
## 保險 - 產險
    property_insurance_product = {}
    property_insurance_valid_insurance_num = list(collection_name.aggregate(property_insurance_valid_insurance_num_pipeline))
    ### 如果完全沒有符合的 document，則跳過
    try:
        property_insurance_product['valid_insurance_num'] = property_insurance_valid_insurance_num[0]['valid_insurance_num']
    except IndexError:
        property_insurance_product['valid_insurance_num'] = 0
    insurance_product_list=[]
    insurance_product_list= insurance_product_list + list(collection_name.aggregate(property_insurance_payment_type_pipeline))
    insurance_product_list= insurance_product_list + list(collection_name.aggregate(property_insurance_purchase_channel_pipeline))
    # grouping
    grouped_data = defaultdict(dict)
    for item in insurance_product_list:
        id_value = item['_id']
        item.pop('_id', None)  # 删除'i鍵，防止覆蓋
        grouped_data[id_value].update(item)
    insurance_product_list = [{'insurance_product_id': key, **value} for key, value in grouped_data.items()]
    property_insurance_product['insurance_product_list']= insurance_product_list
    # insurance 合併 #
    insurance_product_info ={}
    insurance_product_info['life_insurance_product'] = life_insurance_product
    insurance_product_info['property_insurance_product'] = property_insurance_product

## 數據匯總
    summary_customer_quantity_result = list(collection_name.aggregate(summary_pipeline))
    ### 如果完全沒有符合的 document，則跳過
    try:
        summary_customer_quantity = summary_customer_quantity_result[0]['summary_customer_quantity']
    except IndexError:
        summary_customer_quantity = {'unsecured_loan': 0, 'secured_loan': 0, 'secured_loan_insurance': 0, 'secured_loan_bank': 0, 'life_insurance': 0, 'property_insurance': 0}


    return customer_quantity_sum, \
        age_list, gender_list, age_gender_list, residence_list, \
        annual_income_list, annual_income_low_list,  annual_income_high_list, \
        platform_list, bank_account_type_list,  investment_product_list, \
        credit_card_number_list, credit_card_limit_list, \
        unsecured_loan_product_list, secured_loan_product, \
        insurance_product_info, summary_customer_quantity

## 產品條件設定 def get_product_condition

In [ ]:
def get_product_condition(type_id, product_id):
    if type_id == 1:
        condition = {"loan_product_info.unsecured_loan_product_list.loan_product_id": product_id}
    elif type_id == 2:
        condition = {"loan_product_info.secured_loan_product_list.bank_loan_product_list.loan_product_id": product_id}
    elif type_id == 3:
        condition = {"loan_product_info.secured_loan_product_list.life_insurance_loan_product_list.loan_product_id": product_id}
    elif type_id == 4:
        condition = {"insurance_product_info.life_insurance_product.insurance_product_list.insurance_product_id": product_id}
    elif type_id == 5:
        condition = {"insurance_product_info.property_insurance_product.insurance_product_list.insurance_product_id": product_id}
    elif type_id == 6:
        condition = {"bank_account_type_list": product_id}
    elif type_id == 7:
        condition = {"credit_card_info": {"$exists": True}}
    elif type_id == 8:
        condition = {"investment_product_list": product_id}
    elif type_id == 9:
        if product_id == 1:
            condition = {"platform_activity_status.platform_id": {"$in": [1]}}
        elif product_id == 2:
            condition = {"platform_activity_status.platform_id": {"$in": [2, 3]}}
        elif product_id == 3:
            condition = {"platform_activity_status.platform_id": {"$in": [4, 5]}}
        elif product_id == 4:
            condition = {"platform_activity_status.platform_id": {"$in": [6, 7, 8, 9, 10]}}
        elif product_id == 5:
            condition = {"platform_activity_status.platform_id": {"$in": [11, 12]}}
        else:
            condition = None
    else:
        condition = None

    return condition

## 文氏圖的情境一：一間公司

In [ ]:
## 文氏圖的情境一：一間公司
def one_company():
  s_time = time.time()
  count = 0
  ################################################# 產生組合並迭代 #################################################
  for company, identity, status in product(company_count, identity_count, status_count):
      print(f"公司：{company}，身份：{identity}，狀態：{status}")
      condition = {
          "customer_source": company,
          "customer_identity": identity,
          "customer_status": status
      }

      # 如果是 人壽一般活躍戶 或 產險一般活躍戶 則跳過（沒定義）
      if company == 2 and identity == 1 and status == 2:
          continue
      if company == 3 and identity == 1 and status == 2:
          continue

      condition_code = {
          # 篩選符合條件的 doc，才會進入下一階段處理
          "$match": {
              # and: 可以再加上其他條件
              "$and": [
                  {
                      "customer_source_list": {
                          "$elemMatch": condition
                      }
                  }
              ]
          }
      }

      customer_quantity_sum, age_list, gender_list, age_gender_list, residence_list, \
          annual_income_list, annual_income_low_list,  annual_income_high_list, \
          platform_list, bank_account_type_list, investment_product_list, \
          credit_card_number_list, credit_card_limit_list, \
          unsecured_loan_product_list, secured_loan_product, insurance_product_info, \
          summary_customer_quantity = aggregate_data(collection_name)

  ################################################# 寫入 AGGR表 #################################################

      # condition_list 是用來寫入 customer_aggr 的 customer_source_list (只有產品就不用建立此list).
      # condition 是前面公司部分篩選所下的條件
      condition_list = [{}]
      condition_list[0]["customer_source_id"] = condition['customer_source']
      condition_list[0]["customer_identity"] = condition['customer_identity']
      condition_list[0]["customer_status"] = condition['customer_status']


      # 建立 customer_aggr 並加入資料
      customer_aggr = {}

      ## 篩選器資訊 亦有可能為 null (也就是直接刪除該欄位，沒有寫入)
      customer_aggr["calculate_month"] = calculate_month
      customer_aggr["customer_source_list"] = condition_list

      customer_aggr["customer_quantity"] = customer_quantity_sum
      customer_aggr["gender_list"] = gender_list
      customer_aggr["residence_list"] = residence_list
      customer_aggr["age_list"] = age_list
      customer_aggr["annual_income_list"] = annual_income_list
      customer_aggr["annual_income_low_list"] = annual_income_low_list
      customer_aggr["annual_income_high_list"] = annual_income_high_list
      customer_aggr["age_gender_list"] = age_gender_list
      customer_aggr["platform_list"] = platform_list
      customer_aggr["bank_account_type_list"] = bank_account_type_list
      customer_aggr["investment_product_list"] = investment_product_list
      customer_aggr["credit_card_number_list"] = credit_card_number_list
      customer_aggr["credit_card_limit_list"] = credit_card_limit_list
      customer_aggr["unsecured_loan_product_list"] = unsecured_loan_product_list
      customer_aggr["secured_loan_product"] = secured_loan_product
      customer_aggr["insurance_product_info"] = insurance_product_info
      customer_aggr["summary_customer_quantity"] = summary_customer_quantity

      # 寫入 customer_aggr collection
      customer_aggr_collection = db[GDP_CUSTOMER_AGGREGATION]     ## 正式環境：GDP_CUSTOMER_AGGR.      ## 測試環境：ALAN_CUSTOMER_AGGR
      customer_aggr_collection.insert_one(customer_aggr)
      print("inserted to customer_aggr collection")

      count += 1

  e_time = time.time()
  print(f"{round(e_time - s_time, 2)} seconds in total.")
  print(f"{count} document created.")

  return 0


In [ ]:
## 文氏圖的情境一：一間公司
s_time = time.time()
count = 0
################################################# 產生組合並迭代 #################################################
for company, identity, status in product(company_count, identity_count, status_count):
    print(f"公司：{company}，身份：{identity}，狀態：{status}")
    condition = {
        "customer_source": company,
        "customer_identity": identity,
        "customer_status": status
    }

    # 如果是 人壽一般活躍戶 或 產險一般活躍戶 則跳過（沒定義）
    if company == 2 and identity == 1 and status == 2:
        continue
    if company == 3 and identity == 1 and status == 2:
        continue

    condition_code = {
        # 篩選符合條件的 doc，才會進入下一階段處理
        "$match": {
            # and: 可以再加上其他條件
            "$and": [
                {
                    "customer_source_list": {
                        "$elemMatch": condition
                    }
                }
            ]
        }
    }

    customer_quantity_sum, age_list, gender_list, age_gender_list, residence_list, \
        annual_income_list, annual_income_low_list,  annual_income_high_list, \
        platform_list, bank_account_type_list, investment_product_list, \
        credit_card_number_list, credit_card_limit_list, \
        unsecured_loan_product_list, secured_loan_product, insurance_product_info, \
        summary_customer_quantity = aggregate_data(collection_name)

################################################# 寫入 AGGR表 #################################################

    # condition_list 是用來寫入 customer_aggr 的 customer_source_list (只有產品就不用建立此list).
    # condition 是前面公司部分篩選所下的條件
    condition_list = [{}]
    condition_list[0]["customer_source_id"] = condition['customer_source']
    condition_list[0]["customer_identity"] = condition['customer_identity']
    condition_list[0]["customer_status"] = condition['customer_status']


    # 建立 customer_aggr 並加入資料
    customer_aggr = {}

    ## 篩選器資訊 亦有可能為 null (也就是直接刪除該欄位，沒有寫入)
    customer_aggr["calculate_month"] = calculate_month
    customer_aggr["customer_source_list"] = condition_list

    customer_aggr["customer_quantity"] = customer_quantity_sum
    customer_aggr["gender_list"] = gender_list
    customer_aggr["residence_list"] = residence_list
    customer_aggr["age_list"] = age_list
    customer_aggr["annual_income_list"] = annual_income_list
    customer_aggr["annual_income_low_list"] = annual_income_low_list
    customer_aggr["annual_income_high_list"] = annual_income_high_list
    customer_aggr["age_gender_list"] = age_gender_list
    customer_aggr["platform_list"] = platform_list
    customer_aggr["bank_account_type_list"] = bank_account_type_list
    customer_aggr["investment_product_list"] = investment_product_list
    customer_aggr["credit_card_number_list"] = credit_card_number_list
    customer_aggr["credit_card_limit_list"] = credit_card_limit_list
    customer_aggr["unsecured_loan_product_list"] = unsecured_loan_product_list
    customer_aggr["secured_loan_product"] = secured_loan_product
    customer_aggr["insurance_product_info"] = insurance_product_info
    customer_aggr["summary_customer_quantity"] = summary_customer_quantity

    # 寫入 customer_aggr collection
    customer_aggr_collection = db[GDP_CUSTOMER_AGGREGATION]     ## 正式環境：GDP_CUSTOMER_AGGR.      ## 測試環境：ALAN_CUSTOMER_AGGR
    customer_aggr_collection.insert_one(customer_aggr)
    print("inserted to customer_aggr collection")

    count += 1

e_time = time.time()
print(f"{round(e_time - s_time, 2)} seconds in total.")
print(f"{count} document created.")

公司：1，身份：1，狀態：1
inserted to customer_aggr collection
公司：1，身份：1，狀態：2
inserted to customer_aggr collection
公司：1，身份：1，狀態：3
inserted to customer_aggr collection
公司：1，身份：2，狀態：1
inserted to customer_aggr collection
公司：1，身份：2，狀態：2
inserted to customer_aggr collection
公司：1，身份：2，狀態：3
inserted to customer_aggr collection
公司：2，身份：1，狀態：1
inserted to customer_aggr collection
公司：2，身份：1，狀態：2
公司：2，身份：1，狀態：3
inserted to customer_aggr collection
公司：2，身份：2，狀態：1
inserted to customer_aggr collection
公司：2，身份：2，狀態：2
inserted to customer_aggr collection
公司：2，身份：2，狀態：3
inserted to customer_aggr collection
公司：3，身份：1，狀態：1
inserted to customer_aggr collection
公司：3，身份：1，狀態：2
公司：3，身份：1，狀態：3
inserted to customer_aggr collection
公司：3，身份：2，狀態：1
inserted to customer_aggr collection
公司：3，身份：2，狀態：2
inserted to customer_aggr collection
公司：3，身份：2，狀態：3
inserted to customer_aggr collection
公司：4，身份：1，狀態：1
inserted to customer_aggr collection
公司：4，身份：1，狀態：2
inserted to customer_aggr collection
公司：4，身份：1，狀態：3
inserted to custome

## 文氏圖的情境二：一個產品

In [ ]:
## 文氏圖的情境二：一個產品
s_time = time.time()
count = 0
################################################# 產生組合並迭代 #################################################
for product_info in products_info:
    type_id = product_info["type_id"]
    product_ids = product_info["product_id"]
    for product_id in product_ids:
        condition = get_product_condition(type_id, product_id)
        print(condition)

        condition_code = {
            # 篩選符合條件的 doc，才會進入下一階段處理
            "$match": condition
        }

        customer_quantity_sum, age_list, gender_list, age_gender_list, residence_list, \
            annual_income_list, annual_income_low_list,  annual_income_high_list, \
            platform_list, bank_account_type_list, investment_product_list, \
            credit_card_number_list, credit_card_limit_list, \
            unsecured_loan_product_list, secured_loan_product, insurance_product_info, \
            summary_customer_quantity = aggregate_data(collection_name)

################################################# 寫入 AGGR表 #################################################
        # 建立 customer_aggr 並加入資料
        customer_aggr = {}

        ## 篩選器資訊 亦有可能為 null (也就是直接刪除該欄位，沒有寫入)
        customer_aggr["calculate_month"] = calculate_month
        #customer_aggr["customer_source_list"] = condition_list
        if type_id == 7:
            customer_aggr["customer_product_type"] = type_id
        else:
            customer_aggr["customer_product_type"] = type_id
            customer_aggr["customer_product_id"] = product_id

        customer_aggr["customer_quantity"] = customer_quantity_sum
        customer_aggr["gender_list"] = gender_list
        customer_aggr["residence_list"] = residence_list
        customer_aggr["age_list"] = age_list
        customer_aggr["annual_income_list"] = annual_income_list
        customer_aggr["annual_income_low_list"] = annual_income_low_list
        customer_aggr["annual_income_high_list"] = annual_income_high_list
        customer_aggr["age_gender_list"] = age_gender_list
        customer_aggr["platform_list"] = platform_list
        customer_aggr["bank_account_type_list"] = bank_account_type_list
        customer_aggr["investment_product_list"] = investment_product_list
        customer_aggr["credit_card_number_list"] = credit_card_number_list
        customer_aggr["credit_card_limit_list"] = credit_card_limit_list
        customer_aggr["unsecured_loan_product_list"] = unsecured_loan_product_list
        customer_aggr["secured_loan_product"] = secured_loan_product
        customer_aggr["insurance_product_info"] = insurance_product_info
        customer_aggr["summary_customer_quantity"] = summary_customer_quantity

        # 寫入 customer_aggr collection
        customer_aggr_collection = db[GDP_CUSTOMER_AGGREGATION]     ## 正式環境：GDP_CUSTOMER_AGGR.      ## 測試環境：ALAN_CUSTOMER_AGGR
        customer_aggr_collection.insert_one(customer_aggr)
        print("inserted to customer_aggr collection")

        count += 1

e_time = time.time()
print(f"{round(e_time - s_time, 2)} seconds in total.")
print(f"{count} document created.")

{'loan_product_info.unsecured_loan_product_list.loan_product_id': 101}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 102}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 103}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 104}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 105}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 106}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 107}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 108}
inserted to customer_aggr collection
{'loan_product_info.unsecured_loan_product_list.loan_product_id': 109}
inserted to customer_aggr collection
{'loan_product_info.unsecure

## 文氏圖的情境三：兩間公司

In [ ]:
## 文氏圖的情境三：兩間公司
s_time = time.time()
count = 0
################################################# 產生組合並迭代 #################################################
for (company1, identity1, status1), (company2, identity2, status2) in product(product(company_count, identity_count, status_count), repeat=2):
    # 檢查組合的順序
    if (company1, identity1, status1) < (company2, identity2, status2):
        print(f"公司{company1}，身份：{identity1}，狀態：{status1}；公司{company2}，身份：{identity2}，狀態：{status2}")

        condition1 = {
            "customer_source": company1,
            "customer_identity": identity1,
            "customer_status": status1
        }
        condition2 = {
            "customer_source": company2,
            "customer_identity": identity2,
            "customer_status": status2
        }

        # 如果是 人壽一般活躍戶 或 產險一般活躍戶 則跳過（沒定義）
        if company1 == 2 and identity1 == 1 and status1 == 2:
            continue
        if company1 == 3 and identity1 == 1 and status1 == 2:
            continue
        if company2 == 2 and identity2 == 1 and status2 == 2:
            continue
        if company2 == 3 and identity2 == 1 and status2 == 2:
            continue

        condition_code = {
            "$match": {
                "$and": [
                    {
                        "customer_source_list": {
                            "$elemMatch": condition1
                        }
                    },
                    {
                        "customer_source_list": {
                            "$elemMatch": condition2
                        }
                    }
                ]
            }
        }

        customer_quantity_sum, age_list, gender_list, age_gender_list, residence_list, \
            annual_income_list, annual_income_low_list,  annual_income_high_list, \
            platform_list, bank_account_type_list, investment_product_list, \
            credit_card_number_list, credit_card_limit_list, \
            unsecured_loan_product_list, secured_loan_product, insurance_product_info, \
            summary_customer_quantity = aggregate_data(collection_name)

################################################# 寫入 AGGR表 #################################################
        # condition_list 是用來寫入 customer_aggr 的 customer_source_list (只有產品就不用建立此list).
        # condition 是前面公司部分篩選所下的條件
        condition_list = [{}, {}]
        condition_list[0]["customer_source_id"] = condition1['customer_source']
        condition_list[0]["customer_identity"] = condition1['customer_identity']
        condition_list[0]["customer_status"] = condition1['customer_status']
        condition_list[1]["customer_source_id"] = condition2['customer_source']
        condition_list[1]["customer_identity"] = condition2['customer_identity']
        condition_list[1]["customer_status"] = condition2['customer_status']

        # 建立 customer_aggr 並加入資料
        customer_aggr = {}

        ## 篩選器資訊 亦有可能為 null (也就是直接刪除該欄位，沒有寫入)
        customer_aggr["calculate_month"] = calculate_month
        customer_aggr["customer_source_list"] = condition_list
        #customer_aggr["customer_product_type"] = type_id
        #customer_aggr["customer_product_id"] = product_id

        customer_aggr["customer_quantity"] = customer_quantity_sum
        customer_aggr["gender_list"] = gender_list
        customer_aggr["residence_list"] = residence_list
        customer_aggr["age_list"] = age_list
        customer_aggr["annual_income_list"] = annual_income_list
        customer_aggr["annual_income_low_list"] = annual_income_low_list
        customer_aggr["annual_income_high_list"] = annual_income_high_list
        customer_aggr["age_gender_list"] = age_gender_list
        customer_aggr["platform_list"] = platform_list
        customer_aggr["bank_account_type_list"] = bank_account_type_list
        customer_aggr["investment_product_list"] = investment_product_list
        customer_aggr["credit_card_number_list"] = credit_card_number_list
        customer_aggr["credit_card_limit_list"] = credit_card_limit_list
        customer_aggr["unsecured_loan_product_list"] = unsecured_loan_product_list
        customer_aggr["secured_loan_product"] = secured_loan_product
        customer_aggr["insurance_product_info"] = insurance_product_info
        customer_aggr["summary_customer_quantity"] = summary_customer_quantity

        # 寫入 customer_aggr collection
        customer_aggr_collection = db[GDP_CUSTOMER_AGGREGATION]     ## 正式環境：GDP_CUSTOMER_AGGR.      ## 測試環境：ALAN_CUSTOMER_AGGR
        customer_aggr_collection.insert_one(customer_aggr)
        print("inserted to customer_aggr collection")

        count += 1

e_time = time.time()
print(f"{round(e_time - s_time, 2)} seconds in total.")
print(f"{count} document created.")

公司1，身份：1，狀態：1；公司1，身份：1，狀態：2
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：3
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：2，狀態：1
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：2，狀態：2
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：2，狀態：3
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司2，身份：1，狀態：1
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司2，身份：1，狀態：2
公司1，身份：1，狀態：1；公司2，身份：1，狀態：3
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司2，身份：2，狀態：1
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司2，身份：2，狀態：2
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司2，身份：2，狀態：3
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司3，身份：1，狀態：1
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司3，身份：1，狀態：2
公司1，身份：1，狀態：1；公司3，身份：1，狀態：3
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司3，身份：2，狀態：1
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司3，身份：2，狀態：2
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司3，身份：2，狀態：3
insert

## 文氏圖的情境四：一個產品 + 一間公司

In [ ]:
## 文氏圖的情境四：一個產品 + 一間公司
s_time = time.time()
count = 0
################################################# 產生組合並迭代 #################################################
for product_info, (company, identity, status) in product(products_info, product(company_count, identity_count, status_count)):
    type_id = product_info["type_id"]
    for product_id in product_info["product_id"]:
        print(f"公司: {company}, 身份: {identity}, 狀態: {status}, 產品類型: {type_id}, 產品ID: {product_id}")
        condition = get_product_condition(type_id, product_id)
        condition2 = {
            "customer_source": company,
            "customer_identity": identity,
            "customer_status": status
        }

        # 如果是 人壽一般活躍戶 或 產險一般活躍戶 則跳過（沒定義）
        if company == 2 and identity == 1 and status == 2:
            continue
        if company == 3 and identity == 1 and status == 2:
            continue

        condition_code = {
            "$match": {
                "$and": [
                    condition,
                    {
                        "customer_source_list": {
                            "$elemMatch": condition2
                        }
                    }
                ]
            }
        }

################################################# 寫入 AGGR表 #################################################
        # condition_list 是用來寫入 customer_aggr 的 customer_source_list (只有產品就不用建立此list).
        # condition 是前面公司部分篩選所下的條件
        condition_list = [{}]
        condition_list[0]["customer_source_id"] = condition2['customer_source']
        condition_list[0]["customer_identity"] = condition2['customer_identity']
        condition_list[0]["customer_status"] = condition2['customer_status']

        # 建立 customer_aggr 並加入資料
        customer_aggr = {}

        ## 篩選器資訊 亦有可能為 null (也就是直接刪除該欄位，沒有寫入)
        customer_aggr["calculate_month"] = calculate_month
        customer_aggr["customer_source_list"] = condition_list
        if type_id == 7:
            customer_aggr["customer_product_type"] = type_id
        else:
            customer_aggr["customer_product_type"] = type_id
            customer_aggr["customer_product_id"] = product_id

        customer_aggr["customer_quantity"] = customer_quantity_sum
        customer_aggr["gender_list"] = gender_list
        customer_aggr["residence_list"] = residence_list
        customer_aggr["age_list"] = age_list
        customer_aggr["annual_income_list"] = annual_income_list
        customer_aggr["annual_income_low_list"] = annual_income_low_list
        customer_aggr["annual_income_high_list"] = annual_income_high_list
        customer_aggr["age_gender_list"] = age_gender_list
        customer_aggr["platform_list"] = platform_list
        customer_aggr["bank_account_type_list"] = bank_account_type_list
        customer_aggr["investment_product_list"] = investment_product_list
        customer_aggr["credit_card_number_list"] = credit_card_number_list
        customer_aggr["credit_card_limit_list"] = credit_card_limit_list
        customer_aggr["unsecured_loan_product_list"] = unsecured_loan_product_list
        customer_aggr["secured_loan_product"] = secured_loan_product
        customer_aggr["insurance_product_info"] = insurance_product_info
        customer_aggr["summary_customer_quantity"] = summary_customer_quantity

        # 寫入 customer_aggr collection
        customer_aggr_collection = db[GDP_CUSTOMER_AGGREGATION]     ## 正式環境：GDP_CUSTOMER_AGGR.      ## 測試環境：ALAN_CUSTOMER_AGGR
        customer_aggr_collection.insert_one(customer_aggr)
        print("inserted to customer_aggr collection")

        count += 1

e_time = time.time()
print(f"{round(e_time - s_time, 2)} seconds in total.")
print(f"{count} document created.")

公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 101
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 102
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 103
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 104
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 105
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 106
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 107
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 108
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 109
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 110
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 111
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 112
inserted to customer_aggr collection
公司: 1, 身份: 1, 狀態: 1, 產品類型: 1, 產品ID: 113
inserted to customer_aggr collection

## 文氏圖的情境五：一個產品 + 兩間公司

In [ ]:
## 文氏圖的情境五：一個產品 + 兩間公司
s_time = time.time()
count = 0
################################################# 產生組合並迭代 #################################################
for product_info, (company1, identity1, status1), (company2, identity2, status2) in product(products_info, product(company_count, identity_count, status_count), product(company_count, identity_count, status_count)):
    type_id = product_info["type_id"]
    for product_id in product_info["product_id"]:
        # 檢查組合的順序
        if (company1, identity1, status1) < (company2, identity2, status2):
            print(f"公司一：{company1}，身份一：{identity1}，狀態一：{status1}；公司二：{company2}，身份二：{identity2}，狀態二：{status2}, 產品類型: {type_id}, 產品ID: {product_id}")

            condition = get_product_condition(type_id, product_id)
            condition1 = {
                "customer_source": company1,
                "customer_identity": identity1,
                "customer_status": status1
            }
            condition2 = {
                "customer_source": company2,
                "customer_identity": identity2,
                "customer_status": status2
            }

            # 如果是 人壽一般活躍戶 或 產險一般活躍戶 則跳過（沒定義）
            if company1 == 2 and identity1 == 1 and status1 == 2:
                continue
            if company1 == 3 and identity1 == 1 and status1 == 2:
                continue
            if company2 == 2 and identity2 == 1 and status2 == 2:
                continue
            if company2 == 3 and identity2 == 1 and status2 == 2:
                continue

            condition_code = {
                "$match": {
                    "$and": [
                        condition,
                        {
                            "customer_source_list": {
                                "$elemMatch": condition1
                            }
                        },
                        {
                            "customer_source_list": {
                                "$elemMatch": condition2
                            }
                        }
                    ]
                }
            }





################################################# 寫入 AGGR表 #################################################
            # condition_list 是用來寫入 customer_aggr 的 customer_source_list (只有產品就不用建立此list).
            condition_list = [{}, {}]
            condition_list[0]["customer_source_id"] = condition1['customer_source']
            condition_list[0]["customer_identity"] = condition1['customer_identity']
            condition_list[0]["customer_status"] = condition1['customer_status']
            condition_list[1]["customer_source_id"] = condition2['customer_source']
            condition_list[1]["customer_identity"] = condition2['customer_identity']
            condition_list[1]["customer_status"] = condition2['customer_status']

            # 建立 customer_aggr 並加入資料
            customer_aggr = {}

            ## 篩選器資訊 亦有可能為 null (也就是直接刪除該欄位，沒有寫入)
            customer_aggr["calculate_month"] = calculate_month
            customer_aggr["customer_source_list"] = condition_list
            if type_id == 7:
                customer_aggr["customer_product_type"] = type_id
            else:
                customer_aggr["customer_product_type"] = type_id
                customer_aggr["customer_product_id"] = product_id

            customer_aggr["customer_quantity"] = customer_quantity_sum
            customer_aggr["gender_list"] = gender_list
            customer_aggr["residence_list"] = residence_list
            customer_aggr["age_list"] = age_list
            customer_aggr["annual_income_list"] = annual_income_list
            customer_aggr["annual_income_low_list"] = annual_income_low_list
            customer_aggr["annual_income_high_list"] = annual_income_high_list
            customer_aggr["age_gender_list"] = age_gender_list
            customer_aggr["platform_list"] = platform_list
            customer_aggr["bank_account_type_list"] = bank_account_type_list
            customer_aggr["investment_product_list"] = investment_product_list
            customer_aggr["credit_card_number_list"] = credit_card_number_list
            customer_aggr["credit_card_limit_list"] = credit_card_limit_list
            customer_aggr["unsecured_loan_product_list"] = unsecured_loan_product_list
            customer_aggr["secured_loan_product"] = secured_loan_product
            customer_aggr["insurance_product_info"] = insurance_product_info
            customer_aggr["summary_customer_quantity"] = summary_customer_quantity

            # 寫入 customer_aggr collection
            customer_aggr_collection = db[GDP_CUSTOMER_AGGREGATION]     ## 正式環境：GDP_CUSTOMER_AGGR.      ## 測試環境：ALAN_CUSTOMER_AGGR
            customer_aggr_collection.insert_one(customer_aggr)
            print("inserted to customer_aggr collection")

            count += 1

e_time = time.time()
print(f"{round(e_time - s_time, 2)} seconds in total.")
print(f"{count} document created.")

公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 101
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 102
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 103
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 104
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 105
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 106
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 107
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 108
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 109
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 110
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 111
inserted to customer_aggr collection
公司1，身份：1，狀態：1；公司1，身份：1，狀態：2, 產品類型: 1, 產品ID: 112
inserted to custo

OperationFailure: you are over your space quota, using 513 MB of 512 MB, full error: {'ok': 0, 'errmsg': 'you are over your space quota, using 513 MB of 512 MB', 'code': 8000, 'codeName': 'AtlasError'}